# **NYC Taxi Trips**

Os **dados** importados abaixo são relacionados à corridas de **Taxi** em **New York** entre **2009** e **2012**.

Os **dados** serão importados e também adicionados em um Banco de Dados. Assim podemos trabalhar com os dados em um Data Frame ou quando são recuperados de um Banco de Dados.

Todos os exercícios possuem uma solução utilizando o **DataFrame** e utilizando **SQL Query**.

Existem 3 classes **(ImportFile, mark, trips)** no projeto que são essenciais para o funcionamento do código.

In [1]:
# importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmplot 
import folium
import sqlite3
from ImportFile import ImportFile
from trips import Trips
from mark import Mark

Em primeiro lugar, vamos inicializar as variáveis abaixo para **importar os dados**

In [2]:
#Inicializa a tabela
table = []
i = ImportFile(table)

In [4]:
#Cria um DB para a importação dos dados
conn = sqlite3.connect('NYC_Trips_1.db')
c = conn.cursor()

Em seguida devemos ler os arquivos com os **dados das corridas** , importá-los na variável `table` e criar o **DataFrame** com os dados da tabela.

In [5]:
#Leitura dos dados
trips_1 = open('data-sample_data-nyctaxi-trips-2009-json_corrigido.json', 'r')
trips_2 = open('data-sample_data-nyctaxi-trips-2010-json_corrigido.json', 'r')
trips_3 = open('data-sample_data-nyctaxi-trips-2011-json_corrigido.json', 'r')
trips_4 = open('data-sample_data-nyctaxi-trips-2012-json_corrigido.json', 'r')

In [6]:
#Importação dos dados
table = i.importJson(trips_1)
table = i.importJson(trips_2)
table = i.importJson(trips_3)
table = i.importJson(trips_4)

In [7]:
#Colocando os dados em um DataFrame
df = pd.DataFrame(table)

In [8]:
#Verificando as informações do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   vendor_id           object 
 1   pickup_datetime     object 
 2   dropoff_datetime    object 
 3   passenger_count     int64  
 4   trip_distance       float64
 5   pickup_longitude    float64
 6   pickup_latitude     float64
 7   rate_code           object 
 8   store_and_fwd_flag  float64
 9   dropoff_longitude   float64
 10  dropoff_latitude    float64
 11  payment_type        object 
 12  fare_amount         float64
 13  surcharge           float64
 14  tip_amount          float64
 15  tolls_amount        float64
 16  total_amount        float64
dtypes: float64(11), int64(1), object(5)
memory usage: 518.8+ MB


In [9]:
#Importar arquivo Data Payment
df_dp = pd.read_csv("data-payment_lookup-csv.csv") 

#Importar arquivo Data Vendor
df_dv = pd.read_csv("data-vendor_lookup-csv.csv") 

Também devemos exportar os dados do **Data Frame** para tabelas no **Banco de Dados**

Três tabelas serão criadas: 
- **trips**: Contendo os dados das viagens
- **payment**: Contendo os dados do pagamento
- **vendor**: Contendo os dados da empresa responsável pela corrida

In [10]:
#Exportar dados para o SQL
#df.to_sql("trips", conn)
#df_dp.to_sql("payment", conn)
df_dv.to_sql("vendor", conn)

Agora, com os **dados** prontos, podemos responder às seguintes **perguntas**:

### Questão 1

Qual a distância média percorrida por viagens com no **máximo** 2 passageiros?

In [ ]:
#Questao 1
print(df.loc[df['passenger_count'] <=2,'trip_distance'].mean())

In [ ]:
#Questão 01 - SQL Query 
print(pd.read_sql_query("""Select avg(trip_distance) From trips
                            Where passenger_count <= 2""",conn))

### Questão 2

Quais os três **maiores vendors** em quantidade total de dinheiro arrecadado?

In [ ]:
#Questao 2
df_q2 = df.groupby(by='vendor_id', sort=True).sum()
df_q2 = df_q2.sort_values(by = 'fare_amount', ascending=True)
df_q2 = df_q2.merge(df_dv,on='vendor_id')
print(df_q2.loc[:,['name','fare_amount']].head(3))

In [ ]:
#Questao 2 - SQL Query
print(pd.read_sql_query("""Select name, sum(fare_amount) From trips
                                   Inner Join vendor On vendor.vendor_id = trips.vendor_id
                            group by trips.vendor_id
                            order by 2 """,conn))

### Questão 3

Faça um histograma da distribuição mensal, nos 4 anos, de corridas pagas em dinheiro?

In [ ]:
#Questao 3
df_q3 = df.copy()
df_q3['trip'] = 1
df_q3 = df_q3.merge(df_dp,on='payment_type')
df_q3 = df_q3.loc[df_q3['payment_lookup'] == 'Cash']
df_q3['pickup_datetime'] = df_q3['pickup_datetime'].astype("datetime64")
df_q3['year'] = df_q3['pickup_datetime'].dt.year
df_q3['month'] = df_q3['pickup_datetime'].dt.month
df_q3 = df_q3.groupby(by=['month', 'year']).count()
df_q3['trip'].plot.hist(bins=12, alpha=0.5)

In [ ]:
#Questao 3 - SQL Query
df_q3 = pd.read_sql_query("""Select name, sum(fare_amount) From trips
                                   Inner Join vendor On vendor.vendor_id = trips.vendor_id
                            group by trips.vendor_id
                            order by 2 """,conn)
df_q3.plot.hist(bins=12, alpha=0.5)

### Questão 4

Faça um gráfico de **série temporal** contando a quantidade de gorjetas de cada dia, nos
**últimos** 3 meses de 2012

In [ ]:
#Questão 04
df_q4 = df.copy()
df_q4['pickup_datetime'] = df_q4['pickup_datetime'].astype("datetime64")
df_q4['date'] = df_q4['pickup_datetime'].dt.date
start_date_q4 = pd.to_datetime('2012-10-01')
end_date_q4 = pd.to_datetime('2013-01-01')
df_q4 = df_q4.loc[(df_q4['pickup_datetime'] >= start_date_q4) & (df_q4['pickup_datetime'] < end_date_q4)]
t = Trips(df_q4)
df_q4 = t.has_tip('tip_amount')
df_q4 = df_q4.loc[df_q4['tip'] ==1, ['date','tip']]
df_q4.groupby(by=['date']).count().plot(style='-o', figsize=(10,5), grid=True)

In [ ]:
#Questao 3 - SQL Query
df_q4 = pd.read_sql_query("""Select strftime("%m-%Y", pickup_datetime) as month, count(1) From trips
                            where pickup_datetime between '2012-10-01' and '2013-01-01'
                              and tip_amount > 0
                            group by strftime("%d-%m-%Y", pickup_datetime)
                            """,conn)
df_q4.plot(style='-o', figsize=(10,5), grid=True)

### Questão 5

Qual o **tempo médio** das corridas nos dias de **sábado** e **domingo**

In [ ]:
#Questão 05
df_q5 = df.copy()
df_q5['weekend'] = ((pd.DatetimeIndex(df_q5['pickup_datetime']).dayofweek) // 5 == 1).astype(float)
df_q5['pickup_datetime'] = df_q5['pickup_datetime'].astype("datetime64")
df_q5['dropoff_datetime'] = df_q5['dropoff_datetime'].astype("datetime64")
df_q5 = df_q5.loc[df_q5['weekend'] == 1]
df_q5['time_trip'] = (df_q5['dropoff_datetime'] - df_q5['pickup_datetime']) / np.timedelta64(1,'m')
print(df_q5['time_trip'].mean())

In [ ]:
#Questão 05 - SQL Query
print(pd.read_sql_query("""Select avg(Cast ((JulianDay(dropoff_datetime) - JulianDay(pickup_datetime)) * 24 * 60 As Integer)) As Mean From trips
                            where strftime('%w', pickup_datetime ) / 5 == 1
                            """,conn))

### Questão 6

Fazer uma visualização em mapa com **latitude** e **longitude** de **pickups** and **dropoffs** no ano
de **2010**

**PS:** A linha contendo o comando `ny_map` está comentada por motivos de pouca memória. Sorry!

In [ ]:
#Questão 06
df_q6 = df.copy()
df_q6['pickup_datetime'] = df_q6['pickup_datetime'].astype("datetime64")
df_q6['date'] = df_q6['pickup_datetime'].dt.date
start_date_q6 = pd.to_datetime('2010-01-01')
end_date_q6 = pd.to_datetime('2011-01-01')
df_q6 = df_q6.loc[(df_q6['pickup_datetime'] >= start_date_q6) & (df_q6['pickup_datetime'] < end_date_q6)].reset_index()
pickup_latitude_collection = df_q6.loc[:,'pickup_latitude']
pickup_longitude_collection = df_q6.loc[:,'pickup_longitude'] 
dropoff_latitude_collection = df_q6.loc[:,'dropoff_latitude'] 
dropoff_longitude_collection =  df_q6.loc[:,'dropoff_longitude'] 
ny_map = folium.Map([40.6643,-73.9385], zoom_start = 11)
m = Mark(ny_map)
m.insertMarkPickup(pickup_latitude_collection,pickup_longitude_collection)
m.insertMarkDropoff(dropoff_latitude_collection,dropoff_longitude_collection)
#ny_map

In [ ]:
#Questão 06 - SQL Query
df_q6 = pd.read_sql_query("""Select pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude From trips
                            where pickup_datetime between '2010-01-01' and '2011-01-01'
                            """,conn)

ny_map = folium.Map([40.6643,-73.9385], zoom_start = 11)
m = Mark(ny_map)
m.insertMarkPickup(df_q6['pickup_latitude'],df_q6['pickup_longitude'])
m.insertMarkDropoff(df_q6['dropoff_latitude'],df_q6['dropoff_longitude'])
#ny_map